# Interfaces
## Abstract Base Classes (or learning your ABCs)

In [ ]:
from abc import ABC
from abc import abstractmethod


class Animal(ABC):
    @abstractmethod
    def eat(self) -> None:
        ...

In [ ]:
class Squirrel(Animal):
    ...


squirrel_a: Squirrel = Squirrel()

In [ ]:
class Squirrel(Animal):
    def eat(self) -> None:
        print("squirrel's eatin'")


squirrel_b: Animal = Squirrel()
squirrel_b.eat()
isinstance(squirrel_b, Animal)

Using an abstract base class, implementation is checked when objects are initialized, not when the class is created.

Abstract base classes should be used with care. They function similarly to interfaces when all methods are defined with the `@abstractmethod` decorator, but as they rely on inheritance, they come with the same problems if you deviate from this. Additionally, excessive use of them can incur a runtime cost, as each instance of your implementation must also carry around an instance of the base class.

Another thing to note is that python supports multiple-inheritance. This means that an implementation can inherit from multiple base-classes. They are not limited in the same way Java and C# abstract classes are.

## Protocols

In [ ]:
from typing import Protocol
from typing import Literal


Direction = Literal["North", "East", "South", "West"]
class Moveable(Protocol):
    @abstractmethod
    def move(self, direction: Direction) -> bool:
        ...

In [ ]:
def move_moveables(moveables: list[Moveable], direction: Direction) -> bool:
    return all(moveable.move(direction) for moveable in moveables)

In [ ]:
#  class Person(Moveable):
class Person:
    def move(self, direction: Direction) -> bool:
        print(f"moving {direction}")
        return True

move_moveables([Person()], "North")  # no complaints from type checker
move_moveables([1], "South")  # complaints from type checker


Here, we haven't explicitly defined `Person` as an implementation of `Moveable`, but due to *structural subtyping* (also known as duck typing), the `Person` class can still be used as a `Moveable`, and can be statically checked.

No runtime checks are performed on any Protocols unless the you do it explicitly, allowing for runtime checking can be done with the `@runtime_checkable` decorator on the protocol.

The main downside of Protocols are that it is often difficult to know the intention of a class. Is it supposed to implementing a protocol? Was it just a similar name? What if two protocols define the same method name with very different underlying meanings?

Additionally, static typecheckers do not warn you when your class doesn't implement a protocol, the warning instead is `Expected type <Protocol>, got <Your class> instead` on some function call that takes the protocol.

To get around this, a class can explicitly inherit the protocol, in which case the protocol functions identically to an abstract baseclass.

An interesting application of protocols is that if you define one that has the same signature as a built-in type, you may use that type as the protocol.

In [ ]:
class IntList(Protocol):
    def __getitem__(self, index: int) -> int:
        ...
    
    def append(self, val: int) -> None:
        ...

def append_1(myList: IntList) -> IntList:
    myList.append(1)
    return myList


append_1([1,2,3,4])  # no complaints from type checker
append_1(["a", "b"])  # complaints from type checker

If more rigerous checking is required, you can look into zope interfaces, a Python package that provides interface functionality based around decorators. I have not used it myself, but I believe it's more similar to Java than the approaches described above. It is not part of the standard library, so you'd have to install it via pip.